# Codebusters KPIs and Other Usefull Patterns

## import libraries

In [523]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import pygwalker as pyg

Import and clean `all_incident_requests` data set 
- remove unnecessary columns
- rename columns

# Prepare data with all tickets

### Import all incident request

Tickets are taken from the following jira filter:<br/> `project = CI AND issuetype in (standardIssueTypes(), "Expense Delivery") AND "Epic Link" is EMPTY AND "Case Number/s" is not EMPTY AND cf[14125] in ("Service Request (ir)") AND resolved is not EMPTY AND resolutiondate >= 2022-12-19`

In [524]:
all_ir_origin = pd.read_csv('../../DataSets/MainTickets/all_incident_requests.csv')
all_ir_origin

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Project name,Project type,Project lead,Project description,...,Comment.26,Comment.27,Comment.28,Comment.29,Comment.30,Comment.31,Comment.32,Comment.33,Comment.34,Comment.35
0,CMS - Not working on approval process,CYTCI-10248111,7809422,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Porsche AT - Adjust the summer time calculate,CYTCI-9630,8042763,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"rheinmetall - Error for ""error.expense.script...",CYTCI-9627,8041529,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,egis: export configuratioon /CYTCI-9415,CYTCI-9625,8035785,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,daimler: pdf report correction on mileage,CYTCI-9552,7981834,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,"Apleona - Adjust Unicode ""&#x2F"" instead of a ...",CYTCI-6089,7651525,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323,hensoldt - not working SSO link on Approval no...,CYTCI-6059,7651495,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
324,"Allianz - Position ""Werbungskosten""",CYTCI-6058,7651494,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
325,rheinmetall - Approval skipped on ES,CYTCI-5913,7651349,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Utilites functions

In [525]:
def find_escalatetions(row, dataSet):
    label_columns = [col for col in dataSet if 'label' in col.lower()]
    isEscalated = False
    for column in label_columns:
        if 'escalated' in str(row[column]).lower():
            isEscalated = True    
    return isEscalated



def get_sla_limit(row):
    if row['Severity'] == '1 - Critical':
        return 1
    elif row['Severity'] == '2 - Serious' or row['Severity'] == '2 - Serious Urgent':
        return 4
    elif row['Severity'] == '3 - Medium' or row['Severity'] == '3 - Medium Urgent':
        return 10
    elif row['Severity'] == '4 - Minor':
        return 28
    


def verify_sla_limit(row, age_col):
    
    limit = get_sla_limit(row)
    ticket_age = row[age_col].days
    row['overlimit'] = False
    
    if ticket_age > limit:
        row['overlimit'] = True

    return row['overlimit']

# Analyze labels

## Find any escalated ticekt

In [526]:
all_ir_escalated_match = all_ir_origin
all_ir_escalated_match["Is Escalated"] = all_ir_escalated_match.apply(lambda row: find_escalatetions(row, all_ir_escalated_match), axis=1)
all_ir_escalated_match

,Summary,Issue key,Issue id,Issue Type,Status,Project key,Project name,Project type,Project lead,Project description,...,Comment.27,Comment.28,Comment.29,Comment.30,Comment.31,Comment.32,Comment.33,Comment.34,Comment.35,Is Escalated
0,CMS - Not working on approval process,CYTCI-10248111,7809422,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,Porsche AT - Adjust the summer time calculate,CYTCI-9630,8042763,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,"rheinmetall - Error for ""error.expense.script...",CYTCI-9627,8041529,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,egis: export configuratioon /CYTCI-9415,CYTCI-9625,8035785,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,daimler: pdf report correction on mileage,CYTCI-9552,7981834,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,"Apleona - Adjust Unicode ""&#x2F"" instead of a ...",CYTCI-6089,7651525,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
323,hensoldt - not working SSO link on Approval no...,CYTCI-6059,7651495,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
324,"Allianz - Position ""Werbungskosten""",CYTCI-6058,7651494,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
325,rheinmetall - Approval skipped on ES,CYTCI-5913,7651349,Expense Delivery,Closed,CYTCI,codebusters,software,mrohkamm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [527]:
contains_sev = all_ir_escalated_match.filter(like='Severity', axis=1)
contains_sev


,Custom field (Severity),Custom field (Source ISP Severity),Custom field (Source Severity)
0,3 - Medium,NaN,NaN
1,3 - Medium,NaN,NaN
2,3 - Medium,NaN,NaN
3,3 - Medium Urgent,NaN,NaN
4,3 - Medium,NaN,NaN
...,...,...,...
322,3 - Medium,NaN,NaN
323,3 - Medium,NaN,NaN
324,4 - Minor,NaN,NaN
325,4 - Minor,NaN,NaN


# Filter the data

### Select from the dataset only columns necessary for the further analysis

In [528]:
all_ir = all_ir_escalated_match[['Issue key', 'Issue id', 'Status', 'Reporter', 'Created', 'Due Date', 'Resolved', 'Custom field (Customer/s)','Custom field (Type of Request)', 'Is Escalated', 'Custom field (Severity)', 'Resolution']]
all_ir.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request", "Custom field (Severity)": "Severity"},axis=1, inplace=True)

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\2794658133.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir.rename({"Custom field (Customer/s)": "Customer", "Custom field (Type of Request)":"Type of Request", "Custom field (Severity)": "Severity"},axis=1, inplace=True)


In [529]:
all_ir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Issue key        327 non-null    object
 1   Issue id         327 non-null    int64 
 2   Status           327 non-null    object
 3   Reporter         327 non-null    object
 4   Created          327 non-null    object
 5   Due Date         327 non-null    object
 6   Resolved         327 non-null    object
 7   Customer         327 non-null    object
 8   Type of Request  327 non-null    object
 9   Is Escalated     327 non-null    bool  
 10  Severity         327 non-null    object
 11  Resolution       327 non-null    object
dtypes: bool(1), int64(1), object(10)
memory usage: 28.5+ KB


In [530]:
all_ir.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           0
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
Is Escalated       0
Severity           0
Resolution         0
dtype: int64

### Convert date table to `datetime` object

In [531]:
all_ir["Created"] = pd.to_datetime(all_ir["Created"])
all_ir["Resolved"] = pd.to_datetime(all_ir["Resolved"])

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\119588641.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir["Created"] = pd.to_datetime(all_ir["Created"])
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\119588641.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir["Resolved"] = pd.to_datetime(all_ir["Resolved"])


In [532]:
all_ir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Issue key        327 non-null    object        
 1   Issue id         327 non-null    int64         
 2   Status           327 non-null    object        
 3   Reporter         327 non-null    object        
 4   Created          327 non-null    datetime64[ns]
 5   Due Date         327 non-null    object        
 6   Resolved         327 non-null    datetime64[ns]
 7   Customer         327 non-null    object        
 8   Type of Request  327 non-null    object        
 9   Is Escalated     327 non-null    bool          
 10  Severity         327 non-null    object        
 11  Resolution       327 non-null    object        
dtypes: bool(1), datetime64[ns](2), int64(1), object(8)
memory usage: 28.5+ KB


In [533]:
all_ir.shape

(327, 12)

In [534]:
all_ir["ticket_age"] = all_ir["Resolved"] - all_ir["Created"]
all_ir

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\2921873505.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir["ticket_age"] = all_ir["Resolved"] - all_ir["Created"]


,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,Severity,Resolution,ticket_age
0,CYTCI-10248111,7809422,Closed,yutaka.yasumi,2023-10-02 06:32:00,2023-10-12 00:00,2023-10-19 11:57:00,CMS Hasche Sigle,Incident Request (IR),False,3 - Medium,Rejected,17 days 05:25:00
1,CYTCI-9630,8042763,Closed,yutaka.yasumi,2023-11-27 09:08:00,2023-12-06 00:00,2023-11-29 10:14:00,Porsche,Incident Request (IR),False,3 - Medium,Rejected,2 days 01:06:00
2,CYTCI-9627,8041529,Closed,yutaka.yasumi,2023-11-27 03:51:00,2023-12-07 00:00,2023-11-30 08:27:00,Rheinmetall,Incident Request (IR),False,3 - Medium,Done,3 days 04:36:00
3,CYTCI-9625,8035785,Closed,nsanchezruiz,2023-11-24 12:33:00,2023-11-30 00:00,2023-11-28 15:35:00,Egis,Incident Request (IR),False,3 - Medium Urgent,Done,4 days 03:02:00
4,CYTCI-9552,7981834,Closed,nsanchezruiz,2023-11-14 14:58:00,2023-11-27 00:00,2023-11-29 10:03:00,MFTBC,Incident Request (IR),False,3 - Medium,Done,14 days 19:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,CYTCI-6089,7651525,Closed,yutaka.yasumi,2021-12-06 05:16:00,2021-12-12 00:00,2022-01-17 01:05:00,Apleona,Incident Request (IR),False,3 - Medium,Done,41 days 19:49:00
323,CYTCI-6059,7651495,Closed,yutaka.yasumi,2021-11-25 05:05:00,2021-11-29 00:00,2022-01-17 02:41:00,Hensoldt,Incident Request (IR),False,3 - Medium,Rejected,52 days 21:36:00
324,CYTCI-6058,7651494,Closed,ssanz,2021-11-24 17:21:00,2021-11-29 00:00,2022-02-22 12:55:00,Allianz,Incident Request (IR),False,4 - Minor,Rejected,89 days 19:34:00
325,CYTCI-5913,7651349,Closed,yutaka.yasumi,2021-10-28 13:45:00,2022-01-20 00:00,2022-01-17 11:26:00,Rheinmetall,Incident Request (IR),False,4 - Minor,Rejected,80 days 21:41:00


**Summary**
- there is not ir ticket without `resolved` date
- there is no `null` value in any of the column
- date type columns converted to `datetime` object in order to make calculucations on these columns

# Prepare data with individuals tasks for incident requests

## Import and clean `all_individual_tasks` data set 
- we are not able to downaload full data becuase of jira limitation, therefore we need to combine data from differen periods
- remove unnecessary columns
- replace missing `date` values with 0
- drop rows with "null" values

In [535]:
all_individual_tasks_path = glob.glob("../../DataSets/IndividualTasks/" + "*.csv")


all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename, encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))
all_individual_tasks_combined

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\986537800.py:4: DtypeWarning: Columns (111,196,261,431,489) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename, encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\986537800.py:4: DtypeWarning: Columns (21,26,39,40,41,80,83,94,267,296,306,438,439,497) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename, encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\986537800.py:4: DtypeWarning: Columns (35,36,109,112,262,430,431,432,433,434,435,436,437,438,439,440,441,442,486,500,548,549) have mixed types. Specify dtype option on import or set low_memory=False.
  all_indi

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\986537800.py:4: DtypeWarning: Columns (25,27,28,29,38,39,40,41,42,43,44,45,46,121,127,128,155,203,264,305,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,481,543,544,545,546,547,548,549) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename, encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\986537800.py:4: DtypeWarning: Columns (23,28,37,38,401,402,449,511,512) have mixed types. Specify dtype option on import or set low_memory=False.
  all_individual_tasks_combined =pd.concat(map(lambda filename: pd.read_csv(filename, encoding='unicode_escape').assign(source=filename), all_individual_tasks_path))


,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,Project lead,...,Inward issue link (Blocks).2,Comment.2,Comment.3,Comment.4,Comment.5,Inward issue link (Blocks).3,Inward issue link (Relates),Def of Done,Lifecycle Items,Comment.6
0,Review,CYTCI-10179980,7679692,7650050,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Clarify,CYTCI-10179977,7679691,7650034,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Explain,CYTCI-10179961,7679689,7650029,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Communicate,CYTCI-10179959,7679687,7649877,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Import in Production,CYTCI-10179958,7679686,7649877,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1263,Configure,CYTCI-10240611,7705909,7654080,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1264,Configure,CYTCI-10239936,7705523,7654026,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1265,Do,CYTCI-10239710,7705402,7654004,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1266,Review,CYTCI-10234353,7703060,7653656,Individual Task,Done,CYTCI,codebusters,software,mrohkamm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Reset indexex of a combined dataset </br>
Indexes of a new dataset are not appropriate, we need to apply `reset_index` method in order to match them with the number of rows

In [536]:
all_individual_tasks_combined.reset_index(inplace=True)
all_individual_tasks_combined

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\4115877976.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  all_individual_tasks_combined.reset_index(inplace=True)


,index,Summary,Issue key,Issue id,Parent id,Issue Type,Status,Project key,Project name,Project type,...,Inward issue link (Blocks).2,Comment.2,Comment.3,Comment.4,Comment.5,Inward issue link (Blocks).3,Inward issue link (Relates),Def of Done,Lifecycle Items,Comment.6
0,0,Review,CYTCI-10179980,7679692,7650050,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Clarify,CYTCI-10179977,7679691,7650034,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Explain,CYTCI-10179961,7679689,7650029,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Communicate,CYTCI-10179959,7679687,7649877,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Import in Production,CYTCI-10179958,7679686,7649877,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33845,1263,Configure,CYTCI-10240611,7705909,7654080,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33846,1264,Configure,CYTCI-10239936,7705523,7654026,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33847,1265,Do,CYTCI-10239710,7705402,7654004,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33848,1266,Review,CYTCI-10234353,7703060,7653656,Individual Task,Done,CYTCI,codebusters,software,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [537]:
all_individual_tasks_combined.to_csv("all_individual_tasks_combined.csv")

### Filter columns and select only the ones needed for further calculations

In [538]:
all_individual_tasks = all_individual_tasks_combined[['Summary','Issue id', 'Parent id', 'Created', 'Resolved']]
all_individual_tasks

,Summary,Issue id,Parent id,Created,Resolved
0,Review,7679692,7650050,2021-02-27 14:46,2021-02-27 14:46
1,Clarify,7679691,7650034,2021-02-26 17:29,2021-02-26 17:29
2,Explain,7679689,7650029,2021-02-26 15:37,2021-02-26 15:37
3,Communicate,7679687,7649877,2021-02-26 15:19,2021-02-26 15:20
4,Import in Production,7679686,7649877,2021-02-26 15:19,2021-02-26 15:19
...,...,...,...,...,...
33845,Configure,7705909,7654080,2023-05-29 12:27,2023-11-17 12:15
33846,Configure,7705523,7654026,2023-05-15 12:39,2023-11-17 13:04
33847,Do,7705402,7654004,2023-05-11 10:58,2023-11-17 12:30
33848,Review,7703060,7653656,2023-03-03 12:53,2023-11-02 08:56


In [539]:
all_individual_tasks[all_individual_tasks['Parent id'] == 461872]

,Summary,Issue id,Parent id,Created,Resolved


In [540]:
all_individual_tasks.shape

(33850, 5)

### Convert date like columns to `datetime` object

In [541]:
all_individual_tasks["Created"] = pd.to_datetime(all_individual_tasks["Created"], dayfirst=True, format='mixed')
all_individual_tasks["Resolved"] = pd.to_datetime(all_individual_tasks["Resolved"], dayfirst=True,  format='mixed')
all_individual_tasks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33850 entries, 0 to 33849
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Summary    33850 non-null  object        
 1   Issue id   33850 non-null  int64         
 2   Parent id  33850 non-null  int64         
 3   Created    33850 non-null  datetime64[ns]
 4   Resolved   33822 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 1.3+ MB


C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\2151095576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks["Created"] = pd.to_datetime(all_individual_tasks["Created"], dayfirst=True, format='mixed')
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\2151095576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks["Resolved"] = pd.to_datetime(all_individual_tasks["Resolved"], dayfirst=True,  format='mixed')


In [542]:
all_individual_tasks['Summary'].value_counts()

Summary
Configure                                                                                   6148
Review                                                                                      5907
Communicate                                                                                 4970
Verify                                                                                      4893
Import to Test System                                                                       4405
Import in Production                                                                        4041
Import to Customer Test System                                                              1189
Import to Staging System                                                                     918
Explain                                                                                      752
Clarify                                                                                      406
Do                    

### Check if there are any `null` values

In [543]:
all_individual_tasks.isnull().sum()

Summary       0
Issue id      0
Parent id     0
Created       0
Resolved     28
dtype: int64

In [544]:
all_individual_tasks[all_individual_tasks["Parent id"] == 461872]

,Summary,Issue id,Parent id,Created,Resolved


### Drop all rows with null value

In [545]:
all_individual_tasks_without_null = all_individual_tasks.dropna()
all_individual_tasks_without_null.isnull().sum()

Summary      0
Issue id     0
Parent id    0
Created      0
Resolved     0
dtype: int64

In [546]:
all_individual_tasks_without_null[all_individual_tasks_without_null["Issue id"] == 461872]

,Summary,Issue id,Parent id,Created,Resolved


### Identify and drop duplicated rows

In [547]:
all_individual_tasks_duplicated_rows = all_individual_tasks_without_null[all_individual_tasks_without_null.duplicated()]
all_individual_tasks_duplicated_rows

,Summary,Issue id,Parent id,Created,Resolved
32582,Review,7828796,7828795,2023-10-05 16:21:00,2023-11-02 14:35:00
33585,Communicate,7929412,7917819,2023-11-02 19:45:00,2023-11-02 19:45:00
33590,Import in Production,7928462,7877598,2023-11-02 15:58:00,2023-11-02 16:02:00
33591,Configure,7928460,7877598,2023-11-02 15:58:00,2023-11-02 15:58:00
33593,Communicate,7928316,7891707,2023-11-02 15:26:00,2023-11-02 15:37:00
...,...,...,...,...,...
33750,Configure,7892900,7811905,2023-10-23 16:34:00,2023-11-02 08:43:00
33763,Configure,7882479,7873215,2023-10-19 14:49:00,2023-11-02 13:55:00
33767,Review,7877599,7877598,2023-10-19 02:05:00,2023-11-02 15:58:00
33776,Communicate,7868731,7866065,2023-10-17 09:06:00,2023-11-02 10:49:00


In [548]:
all_individual_tasks_no_duplicates = all_individual_tasks_without_null.drop_duplicates()
all_individual_tasks_no_duplicates

,Summary,Issue id,Parent id,Created,Resolved
0,Review,7679692,7650050,2021-02-27 14:46:00,2021-02-27 14:46:00
1,Clarify,7679691,7650034,2021-02-26 17:29:00,2021-02-26 17:29:00
2,Explain,7679689,7650029,2021-02-26 15:37:00,2021-02-26 15:37:00
3,Communicate,7679687,7649877,2021-02-26 15:19:00,2021-02-26 15:20:00
4,Import in Production,7679686,7649877,2021-02-26 15:19:00,2021-02-26 15:19:00
...,...,...,...,...,...
33844,Review,7706263,7654140,2023-06-07 10:44:00,2023-11-17 12:29:00
33845,Configure,7705909,7654080,2023-05-29 12:27:00,2023-11-17 12:15:00
33846,Configure,7705523,7654026,2023-05-15 12:39:00,2023-11-17 13:04:00
33847,Do,7705402,7654004,2023-05-11 10:58:00,2023-11-17 12:30:00


**Summary:**

- We are interested only in completed tickets so we have droped rows which contain `null` in `resolved` columns
- Data is filtered and cleaned we are ready for analysis part

# Data Analysis 

# Daterminate how long each ticket was in particular individual task

- calculate time for eacch subtask
- calculate how long each ticket was in review
- calculate how long each ticket was "on codebusters"
- calculate how long each ticket was "outside codebusters"


## Calculate time spent in each of the subtask

In [549]:
all_individual_tasks_time_calculation = all_individual_tasks_no_duplicates
all_individual_tasks_time_calculation["time_in"] = all_individual_tasks_time_calculation["Resolved"] - all_individual_tasks_time_calculation["Created"]

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\1582069957.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks_time_calculation["time_in"] = all_individual_tasks_time_calculation["Resolved"] - all_individual_tasks_time_calculation["Created"]


### List all available statuses of the CI ticket subtask

In [550]:
all_individual_tasks_time_calculation["Summary"].value_counts()

Summary
Configure                                                                                   6134
Review                                                                                      5873
Communicate                                                                                 4958
Verify                                                                                      4878
Import to Test System                                                                       4398
Import in Production                                                                        4037
Import to Customer Test System                                                              1179
Import to Staging System                                                                     918
Explain                                                                                      752
Clarify                                                                                      405
Do                    

## Get the lates `communicate` subtaks date for the particular `Parent id`

In [551]:
all_individual_tasks_communicate_time = all_individual_tasks_no_duplicates
all_individual_tasks_communicate_time = all_individual_tasks_communicate_time[all_individual_tasks_communicate_time['Summary'] == "Communicate"]
all_individual_tasks_communicate_time = all_individual_tasks_communicate_time.sort_values(["Parent id", "Created"], ascending=[True, False])
all_individual_tasks_communicate_time = all_individual_tasks_communicate_time.drop_duplicates(subset="Parent id", keep='first')
all_individual_tasks_communicate_time = all_individual_tasks_communicate_time.filter(items=["Parent id","Created"]).set_index('Parent id')
all_individual_tasks_communicate_time.rename({"Created": "Resolved-communicate"},axis=1, inplace=True)

all_individual_tasks_communicate_time.head()

,Resolved-communicate
Parent id,
335272,2022-10-03 11:14:00
360791,2022-10-07 15:31:00
374097,2022-11-24 16:28:00
376147,2022-10-10 14:19:00
388122,2022-10-20 17:08:00


## Calculate time ticket was in review step

In [552]:
all_individual_tasks_reivew = all_individual_tasks_time_calculation[all_individual_tasks_time_calculation['Summary'] == 'Review']
all_individual_tasks_reivew.rename({"time_in": "time_in_review"},axis=1, inplace=True)
all_individual_tasks_reivew.reset_index(inplace=True)
all_individual_tasks_reivew

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\3555403275.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_individual_tasks_reivew.rename({"time_in": "time_in_review"},axis=1, inplace=True)


,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
0,0,Review,7679692,7650050,2021-02-27 14:46:00,2021-02-27 14:46:00,0 days 00:00:00
1,16,Review,7679672,7650047,2021-02-26 14:28:00,2021-02-26 14:28:00,0 days 00:00:00
2,22,Review,7679664,7650045,2021-02-26 12:46:00,2021-02-26 13:17:00,0 days 00:31:00
3,33,Review,7679647,7650040,2021-02-25 22:41:00,2021-02-25 22:42:00,0 days 00:01:00
4,41,Review,7679633,7649967,2021-02-25 14:22:00,2021-02-25 16:31:00,0 days 02:09:00
...,...,...,...,...,...,...,...
5868,33838,Review,7706389,7654151,2023-06-12 13:30:00,2023-11-17 13:27:00,157 days 23:57:00
5869,33839,Review,7706388,7654150,2023-06-12 13:27:00,2023-11-17 13:25:00,157 days 23:58:00
5870,33840,Review,7706387,7654149,2023-06-12 13:26:00,2023-11-17 13:26:00,158 days 00:00:00
5871,33843,Review,7706264,7654141,2023-06-07 10:49:00,2023-11-17 12:28:00,163 days 01:39:00


In [553]:
all_ir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327 entries, 0 to 326
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Issue key        327 non-null    object         
 1   Issue id         327 non-null    int64          
 2   Status           327 non-null    object         
 3   Reporter         327 non-null    object         
 4   Created          327 non-null    datetime64[ns] 
 5   Due Date         327 non-null    object         
 6   Resolved         327 non-null    datetime64[ns] 
 7   Customer         327 non-null    object         
 8   Type of Request  327 non-null    object         
 9   Is Escalated     327 non-null    bool           
 10  Severity         327 non-null    object         
 11  Resolution       327 non-null    object         
 12  ticket_age       327 non-null    timedelta64[ns]
dtypes: bool(1), datetime64[ns](2), int64(1), object(8), timedelta64[ns](1)
memory us

### Grup tickets by the `Parent id` and sum total time spent in `review` subtask


In [554]:
all_individual_tasks_reivew[all_individual_tasks_reivew["Parent id"] == 441934]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review


In [555]:
all_individual_tasks_reivew_sum = all_individual_tasks_reivew.groupby(by=["Parent id"], dropna=False)["time_in_review"].sum()
all_individual_tasks_reivew_sum = all_individual_tasks_reivew_sum.to_frame()
all_individual_tasks_reivew_sum.reset_index(inplace=True)
all_individual_tasks_reivew_sum

,Parent id,time_in_review
0,336670,0 days 00:01:00
1,393585,3 days 13:22:00
2,399228,52 days 06:54:00
3,402477,0 days 18:56:00
4,404450,11 days 19:36:00
...,...,...
5102,8055536,0 days 00:01:00
5103,8057074,0 days 16:43:00
5104,8060101,0 days 00:03:00
5105,8061206,0 days 00:13:00


In [556]:
all_individual_tasks_reivew_sum[all_individual_tasks_reivew_sum["Parent id"] == 465210]

,Parent id,time_in_review


In [557]:
all_individual_tasks_reivew_sum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5107 entries, 0 to 5106
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype          
---  ------          --------------  -----          
 0   Parent id       5107 non-null   int64          
 1   time_in_review  5107 non-null   timedelta64[ns]
dtypes: int64(1), timedelta64[ns](1)
memory usage: 79.9 KB


In [558]:
all_individual_tasks_reivew_sum[all_individual_tasks_reivew_sum["Parent id"] == 465210]

,Parent id,time_in_review


### Check on the particular ticket if the total time is summed correctly

In [559]:
all_individual_tasks_reivew.groupby("Parent id").count()

,index,Summary,Issue id,Created,Resolved,time_in_review
Parent id,,,,,,
336670,1,1,1,1,1,1
393585,1,1,1,1,1,1
399228,3,3,3,3,3,3
402477,2,2,2,2,2,2
404450,1,1,1,1,1,1
...,...,...,...,...,...,...
8055536,1,1,1,1,1,1
8057074,1,1,1,1,1,1
8060101,1,1,1,1,1,1


In [560]:
all_individual_tasks_reivew.loc[all_individual_tasks_reivew["Parent id"] == 345462]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review


In [561]:
type(all_individual_tasks_reivew_sum)

pandas.core.frame.DataFrame

In [562]:
all_individual_tasks_reivew_sum_df = all_individual_tasks_reivew_sum
all_individual_tasks_reivew_sum_df.reset_index(inplace=True)
all_individual_tasks_reivew_sum_df

,index,Parent id,time_in_review
0,0,336670,0 days 00:01:00
1,1,393585,3 days 13:22:00
2,2,399228,52 days 06:54:00
3,3,402477,0 days 18:56:00
4,4,404450,11 days 19:36:00
...,...,...,...
5102,5102,8055536,0 days 00:01:00
5103,5103,8057074,0 days 16:43:00
5104,5104,8060101,0 days 00:03:00
5105,5105,8061206,0 days 00:13:00


In [563]:
all_individual_tasks_reivew_sum_df.loc[all_individual_tasks_reivew_sum_df["Parent id"] == 345462]

,index,Parent id,time_in_review


In [564]:
all_individual_tasks_reivew_sum_df.isnull().sum()

index             0
Parent id         0
time_in_review    0
dtype: int64

In [565]:
all_individual_tasks_reivew_sum_df

,index,Parent id,time_in_review
0,0,336670,0 days 00:01:00
1,1,393585,3 days 13:22:00
2,2,399228,52 days 06:54:00
3,3,402477,0 days 18:56:00
4,4,404450,11 days 19:36:00
...,...,...,...
5102,5102,8055536,0 days 00:01:00
5103,5103,8057074,0 days 16:43:00
5104,5104,8060101,0 days 00:03:00
5105,5105,8061206,0 days 00:13:00


**Summary:**
- There are some tickets which have been more than once in the `Review` step
- We can notice that after gruping and summing tickets, case with 'Parent id' `345462` has total time equals more than `8 days`, which is the sum of the time spend in previous two `Review` subtasks

### Merge `all_ir` dataset with `all_individual_tasks_reivew_sum_df`

In [566]:
all_ir_review_time = pd.merge(all_ir, all_individual_tasks_reivew_sum_df[['Parent id', 'time_in_review']], how="left",left_on = "Issue id",right_on = "Parent id")
all_ir_review_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,Severity,Resolution,ticket_age,Parent id,time_in_review
0,CYTCI-10248111,7809422,Closed,yutaka.yasumi,2023-10-02 06:32:00,2023-10-12 00:00,2023-10-19 11:57:00,CMS Hasche Sigle,Incident Request (IR),False,3 - Medium,Rejected,17 days 05:25:00,7809422.0,8 days 09:38:00
1,CYTCI-9630,8042763,Closed,yutaka.yasumi,2023-11-27 09:08:00,2023-12-06 00:00,2023-11-29 10:14:00,Porsche,Incident Request (IR),False,3 - Medium,Rejected,2 days 01:06:00,8042763.0,2 days 01:06:00
2,CYTCI-9627,8041529,Closed,yutaka.yasumi,2023-11-27 03:51:00,2023-12-07 00:00,2023-11-30 08:27:00,Rheinmetall,Incident Request (IR),False,3 - Medium,Done,3 days 04:36:00,8041529.0,2 days 06:31:00
3,CYTCI-9625,8035785,Closed,nsanchezruiz,2023-11-24 12:33:00,2023-11-30 00:00,2023-11-28 15:35:00,Egis,Incident Request (IR),False,3 - Medium Urgent,Done,4 days 03:02:00,8035785.0,3 days 04:11:00
4,CYTCI-9552,7981834,Closed,nsanchezruiz,2023-11-14 14:58:00,2023-11-27 00:00,2023-11-29 10:03:00,MFTBC,Incident Request (IR),False,3 - Medium,Done,14 days 19:05:00,7981834.0,1 days 01:23:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,CYTCI-6089,7651525,Closed,yutaka.yasumi,2021-12-06 05:16:00,2021-12-12 00:00,2022-01-17 01:05:00,Apleona,Incident Request (IR),False,3 - Medium,Done,41 days 19:49:00,7651525.0,0 days 09:51:00
323,CYTCI-6059,7651495,Closed,yutaka.yasumi,2021-11-25 05:05:00,2021-11-29 00:00,2022-01-17 02:41:00,Hensoldt,Incident Request (IR),False,3 - Medium,Rejected,52 days 21:36:00,7651495.0,6 days 18:15:00
324,CYTCI-6058,7651494,Closed,ssanz,2021-11-24 17:21:00,2021-11-29 00:00,2022-02-22 12:55:00,Allianz,Incident Request (IR),False,4 - Minor,Rejected,89 days 19:34:00,7651494.0,5 days 21:19:00
325,CYTCI-5913,7651349,Closed,yutaka.yasumi,2021-10-28 13:45:00,2022-01-20 00:00,2022-01-17 11:26:00,Rheinmetall,Incident Request (IR),False,4 - Minor,Rejected,80 days 21:41:00,7651349.0,7 days 20:58:00


In [567]:
all_ir_review_time.isnull().sum()

Issue key           0
Issue id            0
Status              0
Reporter            0
Created             0
Due Date            0
Resolved            0
Customer            0
Type of Request     0
Is Escalated        0
Severity            0
Resolution          0
ticket_age          0
Parent id          50
time_in_review     50
dtype: int64

### Investigate `null` rows after merging  main ir tickets with individual subtasks

In [568]:
null_rows = all_ir_review_time[all_ir_review_time["time_in_review"].isnull()]
null_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 136 to 188
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype          
---  ------           --------------  -----          
 0   Issue key        50 non-null     object         
 1   Issue id         50 non-null     int64          
 2   Status           50 non-null     object         
 3   Reporter         50 non-null     object         
 4   Created          50 non-null     datetime64[ns] 
 5   Due Date         50 non-null     object         
 6   Resolved         50 non-null     datetime64[ns] 
 7   Customer         50 non-null     object         
 8   Type of Request  50 non-null     object         
 9   Is Escalated     50 non-null     bool           
 10  Severity         50 non-null     object         
 11  Resolution       50 non-null     object         
 12  ticket_age       50 non-null     timedelta64[ns]
 13  Parent id        0 non-null      float64        
 14  time_in_review   0 non-null   

In [569]:
null_rows

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,Severity,Resolution,ticket_age,Parent id,time_in_review
136,CYTCI-7932,7653337,Closed,nsanchezruiz,2022-12-27 14:47:00,2023-01-02 00:00,2022-12-27 14:52:00,ProcurX,Incident Request (IR),False,3 - Medium,Rejected,0 days 00:05:00,NaN,NaT
138,CYTCI-7929,7653334,Closed,nsanchezruiz,2022-12-23 10:06:00,2023-01-02 00:00,2022-12-27 10:49:00,Endress Hauser,Incident Request (IR),False,3 - Medium,Rejected,4 days 00:43:00,NaN,NaT
140,CYTCI-7921,7653326,Closed,yutaka.yasumi,2022-12-22 10:09:00,2022-12-25 00:00,2022-12-30 10:21:00,Lufthansa City Center,Incident Request (IR),False,3 - Medium,Done,8 days 00:12:00,NaN,NaT
142,CYTCI-7887,7653292,Closed,nsanchezruiz,2022-12-16 14:40:00,2022-12-20 00:00,2023-01-02 08:59:00,Osram,Incident Request (IR),False,3 - Medium,Done,16 days 18:19:00,NaN,NaT
143,CYTCI-7885,7653290,Closed,nsanchezruiz,2022-12-16 13:35:00,2022-12-21 00:00,2022-12-21 13:51:00,OBB,Incident Request (IR),False,3 - Medium,Done,5 days 00:16:00,NaN,NaT
144,CYTCI-7865,7653270,Closed,nsanchezruiz,2022-12-14 11:30:00,2022-12-19 00:00,2022-12-21 09:17:00,Bentley,Incident Request (IR),False,3 - Medium,Done,6 days 21:47:00,NaN,NaT
145,CYTCI-7861,7653266,Closed,ssanz,2022-12-13 15:15:00,2022-12-19 00:00,2023-03-01 15:56:00,Audi,Incident Request (IR),False,3 - Medium,Done,78 days 00:41:00,NaN,NaT
146,CYTCI-7836,7653242,Closed,yutaka.yasumi,2022-12-07 08:47:00,2022-12-07 00:00,2022-12-16 10:51:00,Endress Hauser,Incident Request (IR),False,3 - Medium,Done,9 days 02:04:00,NaN,NaT
147,CYTCI-7816,7653224,Closed,yutaka.yasumi,2022-12-05 10:09:00,2022-12-08 00:00,2023-03-03 14:03:00,CEPSA,Incident Request (IR),False,3 - Medium,Done,88 days 03:54:00,NaN,NaT
148,CYTCI-7808,7653221,Closed,yutaka.yasumi,2022-12-05 08:05:00,2022-12-08 00:00,2023-01-04 02:16:00,Amadeus,Incident Request (IR),False,3 - Medium,Done,29 days 18:11:00,NaN,NaT


In [570]:
all_ir_review_time_without_null = all_ir_review_time.dropna()
all_ir_review_time_without_null.isnull().sum()

Issue key          0
Issue id           0
Status             0
Reporter           0
Created            0
Due Date           0
Resolved           0
Customer           0
Type of Request    0
Is Escalated       0
Severity           0
Resolution         0
ticket_age         0
Parent id          0
time_in_review     0
dtype: int64

**Summary**:
- Ror tickets like `CI-8488`revew time equals 0, so we need to drop it, because it can leads to wrong KPIs calculations

## Determinate time spent on codebusters

- calculate the total time ticket was in subtasks "Configure", "Review", "mport to Test System", "Import in Production", "Import to Customer Test System"
- apply the results to all tickets dataset

In [571]:
all_individual_with_codebusters_time = all_individual_tasks_time_calculation.loc[all_individual_tasks_time_calculation["Summary"].isin(["Configure", "Review", "Import to Test System", "Import in Production", "Import to Customer Test System", "Import to Test", "Import to Staging System"])]

Summarize time for each codebusters subtask 

In [572]:
all_individual_with_codebusters_time.groupby("Parent id").count()

,Summary,Issue id,Created,Resolved,time_in
Parent id,,,,,
336670,1,1,1,1,1
360791,1,1,1,1,1
376147,2,2,2,2,2
388122,1,1,1,1,1
392021,1,1,1,1,1
...,...,...,...,...,...
8055536,3,3,3,3,3
8057074,1,1,1,1,1
8060101,2,2,2,2,2


In [573]:
all_individual_with_codebusters_time[all_individual_with_codebusters_time['Parent id'] == 329036]

,Summary,Issue id,Parent id,Created,Resolved,time_in


In [574]:
all_individual_with_codebusters_time_sum = all_individual_with_codebusters_time.groupby("Parent id")["time_in"].sum()
all_individual_with_codebusters_time_sum_df = all_individual_with_codebusters_time_sum.to_frame()
all_individual_with_codebusters_time_sum_df.rename({"time_in": "time_on_codebusters"},axis=1, inplace=True)
all_individual_with_codebusters_time_sum_df.reset_index(inplace=True)
all_individual_with_codebusters_time_sum_df

,Parent id,time_on_codebusters
0,336670,0 days 00:01:00
1,360791,0 days 00:01:00
2,376147,0 days 00:00:00
3,388122,28 days 16:13:00
4,392021,232 days 01:49:00
...,...,...
5337,8055536,0 days 00:06:00
5338,8057074,0 days 16:43:00
5339,8060101,0 days 00:04:00
5340,8061206,0 days 01:14:00


In [575]:
# all_individual_tasks_reivew_sum_df = all_individual_tasks_reivew_sum.to_frame()
# all_individual_tasks_reivew_sum_df.reset_index(inplace=True)
# all_individual_tasks_reivew_sum_df

### Merge total time on codebusters with all jira ticekts dataset

In [576]:
all_ir_codebusters_time = pd.merge(all_ir, all_individual_with_codebusters_time_sum_df, how="inner",left_on = "Issue id",right_on = "Parent id")
all_ir_codebusters_time 

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,Severity,Resolution,ticket_age,Parent id,time_on_codebusters
0,CYTCI-10248111,7809422,Closed,yutaka.yasumi,2023-10-02 06:32:00,2023-10-12 00:00,2023-10-19 11:57:00,CMS Hasche Sigle,Incident Request (IR),False,3 - Medium,Rejected,17 days 05:25:00,7809422,8 days 09:38:00
1,CYTCI-9630,8042763,Closed,yutaka.yasumi,2023-11-27 09:08:00,2023-12-06 00:00,2023-11-29 10:14:00,Porsche,Incident Request (IR),False,3 - Medium,Rejected,2 days 01:06:00,8042763,2 days 01:06:00
2,CYTCI-9627,8041529,Closed,yutaka.yasumi,2023-11-27 03:51:00,2023-12-07 00:00,2023-11-30 08:27:00,Rheinmetall,Incident Request (IR),False,3 - Medium,Done,3 days 04:36:00,8041529,2 days 11:27:00
3,CYTCI-9625,8035785,Closed,nsanchezruiz,2023-11-24 12:33:00,2023-11-30 00:00,2023-11-28 15:35:00,Egis,Incident Request (IR),False,3 - Medium Urgent,Done,4 days 03:02:00,8035785,3 days 21:37:00
4,CYTCI-9552,7981834,Closed,nsanchezruiz,2023-11-14 14:58:00,2023-11-27 00:00,2023-11-29 10:03:00,MFTBC,Incident Request (IR),False,3 - Medium,Done,14 days 19:05:00,7981834,12 days 21:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,CYTCI-6089,7651525,Closed,yutaka.yasumi,2021-12-06 05:16:00,2021-12-12 00:00,2022-01-17 01:05:00,Apleona,Incident Request (IR),False,3 - Medium,Done,41 days 19:49:00,7651525,13 days 14:29:00
283,CYTCI-6059,7651495,Closed,yutaka.yasumi,2021-11-25 05:05:00,2021-11-29 00:00,2022-01-17 02:41:00,Hensoldt,Incident Request (IR),False,3 - Medium,Rejected,52 days 21:36:00,7651495,38 days 03:09:00
284,CYTCI-6058,7651494,Closed,ssanz,2021-11-24 17:21:00,2021-11-29 00:00,2022-02-22 12:55:00,Allianz,Incident Request (IR),False,4 - Minor,Rejected,89 days 19:34:00,7651494,89 days 19:35:00
285,CYTCI-5913,7651349,Closed,yutaka.yasumi,2021-10-28 13:45:00,2022-01-20 00:00,2022-01-17 11:26:00,Rheinmetall,Incident Request (IR),False,4 - Minor,Rejected,80 days 21:41:00,7651349,41 days 07:19:00


### Investigate `null` rows after merging  main ir tickets with individual subtasks

In [577]:
all_ir_codebusters_time.isnull().sum()

Issue key              0
Issue id               0
Status                 0
Reporter               0
Created                0
Due Date               0
Resolved               0
Customer               0
Type of Request        0
Is Escalated           0
Severity               0
Resolution             0
ticket_age             0
Parent id              0
time_on_codebusters    0
dtype: int64

In [578]:
all_ir_codebusters_time_without_null = all_ir_codebusters_time.dropna()
all_ir_codebusters_time_without_null.isnull().sum()

Issue key              0
Issue id               0
Status                 0
Reporter               0
Created                0
Due Date               0
Resolved               0
Customer               0
Type of Request        0
Is Escalated           0
Severity               0
Resolution             0
ticket_age             0
Parent id              0
time_on_codebusters    0
dtype: int64

In [579]:
all_ir_codebusters_time_without_null

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,Severity,Resolution,ticket_age,Parent id,time_on_codebusters
0,CYTCI-10248111,7809422,Closed,yutaka.yasumi,2023-10-02 06:32:00,2023-10-12 00:00,2023-10-19 11:57:00,CMS Hasche Sigle,Incident Request (IR),False,3 - Medium,Rejected,17 days 05:25:00,7809422,8 days 09:38:00
1,CYTCI-9630,8042763,Closed,yutaka.yasumi,2023-11-27 09:08:00,2023-12-06 00:00,2023-11-29 10:14:00,Porsche,Incident Request (IR),False,3 - Medium,Rejected,2 days 01:06:00,8042763,2 days 01:06:00
2,CYTCI-9627,8041529,Closed,yutaka.yasumi,2023-11-27 03:51:00,2023-12-07 00:00,2023-11-30 08:27:00,Rheinmetall,Incident Request (IR),False,3 - Medium,Done,3 days 04:36:00,8041529,2 days 11:27:00
3,CYTCI-9625,8035785,Closed,nsanchezruiz,2023-11-24 12:33:00,2023-11-30 00:00,2023-11-28 15:35:00,Egis,Incident Request (IR),False,3 - Medium Urgent,Done,4 days 03:02:00,8035785,3 days 21:37:00
4,CYTCI-9552,7981834,Closed,nsanchezruiz,2023-11-14 14:58:00,2023-11-27 00:00,2023-11-29 10:03:00,MFTBC,Incident Request (IR),False,3 - Medium,Done,14 days 19:05:00,7981834,12 days 21:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,CYTCI-6089,7651525,Closed,yutaka.yasumi,2021-12-06 05:16:00,2021-12-12 00:00,2022-01-17 01:05:00,Apleona,Incident Request (IR),False,3 - Medium,Done,41 days 19:49:00,7651525,13 days 14:29:00
283,CYTCI-6059,7651495,Closed,yutaka.yasumi,2021-11-25 05:05:00,2021-11-29 00:00,2022-01-17 02:41:00,Hensoldt,Incident Request (IR),False,3 - Medium,Rejected,52 days 21:36:00,7651495,38 days 03:09:00
284,CYTCI-6058,7651494,Closed,ssanz,2021-11-24 17:21:00,2021-11-29 00:00,2022-02-22 12:55:00,Allianz,Incident Request (IR),False,4 - Minor,Rejected,89 days 19:34:00,7651494,89 days 19:35:00
285,CYTCI-5913,7651349,Closed,yutaka.yasumi,2021-10-28 13:45:00,2022-01-20 00:00,2022-01-17 11:26:00,Rheinmetall,Incident Request (IR),False,4 - Minor,Rejected,80 days 21:41:00,7651349,41 days 07:19:00


**Summary:**

## Determinate time spent outside codebusters

- calculate the total time ticket was in one of the subtask "Explain", "Verify", "Clarify", "Communicate"
- apply results to all ir tickets dataset

In [580]:
all_individual_others_time = all_individual_tasks_time_calculation.loc[all_individual_tasks_time_calculation["Summary"].isin(["Explain", "Verify", "Clarify", "Communicate"])]
all_individual_others_time 

,Summary,Issue id,Parent id,Created,Resolved,time_in
1,Clarify,7679691,7650034,2021-02-26 17:29:00,2021-02-26 17:29:00,0 days 00:00:00
2,Explain,7679689,7650029,2021-02-26 15:37:00,2021-02-26 15:37:00,0 days 00:00:00
3,Communicate,7679687,7649877,2021-02-26 15:19:00,2021-02-26 15:20:00,0 days 00:01:00
5,Verify,7679685,7649877,2021-02-26 15:19:00,2021-02-26 15:19:00,0 days 00:00:00
9,Clarify,7679681,7649877,2021-02-26 15:18:00,2021-02-26 15:18:00,0 days 00:00:00
...,...,...,...,...,...,...
33815,Communicate,7838831,7826989,2023-10-09 15:07:00,2023-11-13 13:25:00,34 days 22:18:00
33820,Communicate,7709869,7654604,2023-09-27 12:49:00,2023-11-22 15:32:00,56 days 02:43:00
33828,Verify,7709616,7653979,2023-09-19 18:22:00,2023-11-06 09:34:00,47 days 15:12:00
33829,Verify,7709591,7654428,2023-09-19 12:53:00,2023-11-06 14:42:00,48 days 01:49:00


In [581]:
all_individual_others_time[all_individual_others_time["Parent id"] == 461872]

,Summary,Issue id,Parent id,Created,Resolved,time_in


Summarize time for each subtask outside codeubsters

In [582]:
all_individual_others_time_sum = all_individual_others_time.groupby("Parent id")["time_in"].sum()
all_individual_others_time_sum_df = all_individual_others_time_sum.to_frame()
all_individual_others_time_sum_df.rename({"time_in": "time_outside_codebusters"},axis=1, inplace=True)
all_individual_others_time_sum_df.reset_index(inplace=True)
all_individual_others_time_sum_df

,Parent id,time_outside_codebusters
0,335272,0 days 00:00:00
1,336670,3 days 15:01:00
2,338553,0 days 00:01:00
3,360791,0 days 00:00:00
4,374097,0 days 00:01:00
...,...,...
5250,8048691,0 days 00:01:00
5251,8049403,0 days 02:26:00
5252,8050182,0 days 00:00:00
5253,8055536,0 days 00:00:00


In [583]:
all_individual_others_time_sum_df[all_individual_others_time_sum_df['Parent id'] == 461872]

,Parent id,time_outside_codebusters


In [584]:
all_individual_with_codebusters_time_sum_df[all_individual_with_codebusters_time_sum_df["Parent id"] == 461872]

,Parent id,time_on_codebusters


In [585]:
# all_individual_with_codebusters_time_sum = all_individual_with_codebusters_time.groupby("Parent id")["time_in"].sum()
# all_individual_with_codebusters_time_sum_df = all_individual_with_codebusters_time_sum.to_frame()
# all_individual_with_codebusters_time_sum_df.rename({"time_in": "time_on_codebusters"},axis=1, inplace=True)
# all_individual_with_codebusters_time_sum_df.reset_index(inplace=True)
# all_individual_with_codebusters_time_sum_df

### Merge `all_ir` with `all_individual_others_time_sum_df` dataset

In [586]:
all_ir_outside_codebusters_time = pd.merge(all_ir, all_individual_others_time_sum_df, how="inner",left_on = "Issue id",right_on = "Parent id")
all_ir_outside_codebusters_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,Severity,Resolution,ticket_age,Parent id,time_outside_codebusters
0,CYTCI-10248111,7809422,Closed,yutaka.yasumi,2023-10-02 06:32:00,2023-10-12 00:00,2023-10-19 11:57:00,CMS Hasche Sigle,Incident Request (IR),False,3 - Medium,Rejected,17 days 05:25:00,7809422,8 days 19:47:00
1,CYTCI-9630,8042763,Closed,yutaka.yasumi,2023-11-27 09:08:00,2023-12-06 00:00,2023-11-29 10:14:00,Porsche,Incident Request (IR),False,3 - Medium,Rejected,2 days 01:06:00,8042763,0 days 00:00:00
2,CYTCI-9627,8041529,Closed,yutaka.yasumi,2023-11-27 03:51:00,2023-12-07 00:00,2023-11-30 08:27:00,Rheinmetall,Incident Request (IR),False,3 - Medium,Done,3 days 04:36:00,8041529,0 days 17:09:00
3,CYTCI-9625,8035785,Closed,nsanchezruiz,2023-11-24 12:33:00,2023-11-30 00:00,2023-11-28 15:35:00,Egis,Incident Request (IR),False,3 - Medium Urgent,Done,4 days 03:02:00,8035785,0 days 05:24:00
4,CYTCI-9552,7981834,Closed,nsanchezruiz,2023-11-14 14:58:00,2023-11-27 00:00,2023-11-29 10:03:00,MFTBC,Incident Request (IR),False,3 - Medium,Done,14 days 19:05:00,7981834,1 days 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,CYTCI-6089,7651525,Closed,yutaka.yasumi,2021-12-06 05:16:00,2021-12-12 00:00,2022-01-17 01:05:00,Apleona,Incident Request (IR),False,3 - Medium,Done,41 days 19:49:00,7651525,28 days 05:20:00
282,CYTCI-6059,7651495,Closed,yutaka.yasumi,2021-11-25 05:05:00,2021-11-29 00:00,2022-01-17 02:41:00,Hensoldt,Incident Request (IR),False,3 - Medium,Rejected,52 days 21:36:00,7651495,14 days 18:27:00
283,CYTCI-6058,7651494,Closed,ssanz,2021-11-24 17:21:00,2021-11-29 00:00,2022-02-22 12:55:00,Allianz,Incident Request (IR),False,4 - Minor,Rejected,89 days 19:34:00,7651494,0 days 00:00:00
284,CYTCI-5913,7651349,Closed,yutaka.yasumi,2021-10-28 13:45:00,2022-01-20 00:00,2022-01-17 11:26:00,Rheinmetall,Incident Request (IR),False,4 - Minor,Rejected,80 days 21:41:00,7651349,39 days 14:22:00


In [587]:
all_ir_outside_codebusters_time.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
Is Escalated                0
Severity                    0
Resolution                  0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
dtype: int64

**Summary:**

## Combine all created datasets
- In this step we will merge all created erlier datasets: `all_ir_outside_codebusters_time`, `all_ir_codebusters_time` and `all_ir_review_time` 

Merge df containing "review" time with df containg "time on codebusters" 

In [588]:
all_ir_time_on_and_outside_codebusters = pd.merge(all_ir_outside_codebusters_time, all_ir_codebusters_time[['Issue id','time_on_codebusters']],  how="inner",left_on = "Issue id",right_on = "Issue id")
all_ir_time_on_and_outside_codebusters

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,Severity,Resolution,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters
0,CYTCI-10248111,7809422,Closed,yutaka.yasumi,2023-10-02 06:32:00,2023-10-12 00:00,2023-10-19 11:57:00,CMS Hasche Sigle,Incident Request (IR),False,3 - Medium,Rejected,17 days 05:25:00,7809422,8 days 19:47:00,8 days 09:38:00
1,CYTCI-9630,8042763,Closed,yutaka.yasumi,2023-11-27 09:08:00,2023-12-06 00:00,2023-11-29 10:14:00,Porsche,Incident Request (IR),False,3 - Medium,Rejected,2 days 01:06:00,8042763,0 days 00:00:00,2 days 01:06:00
2,CYTCI-9627,8041529,Closed,yutaka.yasumi,2023-11-27 03:51:00,2023-12-07 00:00,2023-11-30 08:27:00,Rheinmetall,Incident Request (IR),False,3 - Medium,Done,3 days 04:36:00,8041529,0 days 17:09:00,2 days 11:27:00
3,CYTCI-9625,8035785,Closed,nsanchezruiz,2023-11-24 12:33:00,2023-11-30 00:00,2023-11-28 15:35:00,Egis,Incident Request (IR),False,3 - Medium Urgent,Done,4 days 03:02:00,8035785,0 days 05:24:00,3 days 21:37:00
4,CYTCI-9552,7981834,Closed,nsanchezruiz,2023-11-14 14:58:00,2023-11-27 00:00,2023-11-29 10:03:00,MFTBC,Incident Request (IR),False,3 - Medium,Done,14 days 19:05:00,7981834,1 days 22:00:00,12 days 21:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,CYTCI-6089,7651525,Closed,yutaka.yasumi,2021-12-06 05:16:00,2021-12-12 00:00,2022-01-17 01:05:00,Apleona,Incident Request (IR),False,3 - Medium,Done,41 days 19:49:00,7651525,28 days 05:20:00,13 days 14:29:00
279,CYTCI-6059,7651495,Closed,yutaka.yasumi,2021-11-25 05:05:00,2021-11-29 00:00,2022-01-17 02:41:00,Hensoldt,Incident Request (IR),False,3 - Medium,Rejected,52 days 21:36:00,7651495,14 days 18:27:00,38 days 03:09:00
280,CYTCI-6058,7651494,Closed,ssanz,2021-11-24 17:21:00,2021-11-29 00:00,2022-02-22 12:55:00,Allianz,Incident Request (IR),False,4 - Minor,Rejected,89 days 19:34:00,7651494,0 days 00:00:00,89 days 19:35:00
281,CYTCI-5913,7651349,Closed,yutaka.yasumi,2021-10-28 13:45:00,2022-01-20 00:00,2022-01-17 11:26:00,Rheinmetall,Incident Request (IR),False,4 - Minor,Rejected,80 days 21:41:00,7651349,39 days 14:22:00,41 days 07:19:00


In [589]:
all_ir_time_on_and_outside_codebusters.isnull().sum()

Issue key                   0
Issue id                    0
Status                      0
Reporter                    0
Created                     0
Due Date                    0
Resolved                    0
Customer                    0
Type of Request             0
Is Escalated                0
Severity                    0
Resolution                  0
ticket_age                  0
Parent id                   0
time_outside_codebusters    0
time_on_codebusters         0
dtype: int64

### Merge `all_tickets_time_on_and_outside_codebusters` with `all_ir_review_time` dataset

In [590]:
all_ir_with_time = pd.merge(all_ir_time_on_and_outside_codebusters, all_ir_review_time[['Issue id','time_in_review']],  how="inner",left_on = "Issue id",right_on = "Issue id")
all_ir_with_time

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,Severity,Resolution,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review
0,CYTCI-10248111,7809422,Closed,yutaka.yasumi,2023-10-02 06:32:00,2023-10-12 00:00,2023-10-19 11:57:00,CMS Hasche Sigle,Incident Request (IR),False,3 - Medium,Rejected,17 days 05:25:00,7809422,8 days 19:47:00,8 days 09:38:00,8 days 09:38:00
1,CYTCI-9630,8042763,Closed,yutaka.yasumi,2023-11-27 09:08:00,2023-12-06 00:00,2023-11-29 10:14:00,Porsche,Incident Request (IR),False,3 - Medium,Rejected,2 days 01:06:00,8042763,0 days 00:00:00,2 days 01:06:00,2 days 01:06:00
2,CYTCI-9627,8041529,Closed,yutaka.yasumi,2023-11-27 03:51:00,2023-12-07 00:00,2023-11-30 08:27:00,Rheinmetall,Incident Request (IR),False,3 - Medium,Done,3 days 04:36:00,8041529,0 days 17:09:00,2 days 11:27:00,2 days 06:31:00
3,CYTCI-9625,8035785,Closed,nsanchezruiz,2023-11-24 12:33:00,2023-11-30 00:00,2023-11-28 15:35:00,Egis,Incident Request (IR),False,3 - Medium Urgent,Done,4 days 03:02:00,8035785,0 days 05:24:00,3 days 21:37:00,3 days 04:11:00
4,CYTCI-9552,7981834,Closed,nsanchezruiz,2023-11-14 14:58:00,2023-11-27 00:00,2023-11-29 10:03:00,MFTBC,Incident Request (IR),False,3 - Medium,Done,14 days 19:05:00,7981834,1 days 22:00:00,12 days 21:05:00,1 days 01:23:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,CYTCI-6089,7651525,Closed,yutaka.yasumi,2021-12-06 05:16:00,2021-12-12 00:00,2022-01-17 01:05:00,Apleona,Incident Request (IR),False,3 - Medium,Done,41 days 19:49:00,7651525,28 days 05:20:00,13 days 14:29:00,0 days 09:51:00
279,CYTCI-6059,7651495,Closed,yutaka.yasumi,2021-11-25 05:05:00,2021-11-29 00:00,2022-01-17 02:41:00,Hensoldt,Incident Request (IR),False,3 - Medium,Rejected,52 days 21:36:00,7651495,14 days 18:27:00,38 days 03:09:00,6 days 18:15:00
280,CYTCI-6058,7651494,Closed,ssanz,2021-11-24 17:21:00,2021-11-29 00:00,2022-02-22 12:55:00,Allianz,Incident Request (IR),False,4 - Minor,Rejected,89 days 19:34:00,7651494,0 days 00:00:00,89 days 19:35:00,5 days 21:19:00
281,CYTCI-5913,7651349,Closed,yutaka.yasumi,2021-10-28 13:45:00,2022-01-20 00:00,2022-01-17 11:26:00,Rheinmetall,Incident Request (IR),False,4 - Minor,Rejected,80 days 21:41:00,7651349,39 days 14:22:00,41 days 07:19:00,7 days 20:58:00


### Merge `all_individual_tasks_communicate_time` with all tikets

In [591]:
all_ir_with_time_communicate = all_ir_with_time.merge(all_individual_tasks_communicate_time, left_on="Issue id", right_on="Parent id", how="left")
all_ir_with_time_communicate.isnull().sum()

Issue key                    0
Issue id                     0
Status                       0
Reporter                     0
Created                      0
Due Date                     0
Resolved                     0
Customer                     0
Type of Request              0
Is Escalated                 0
Severity                     0
Resolution                   0
ticket_age                   0
Parent id                    0
time_outside_codebusters     0
time_on_codebusters          0
time_in_review              10
Resolved-communicate        59
dtype: int64

**Summary**
- there are quite a lot of null values in `Resolved-communicate`, those are the tickets which do not contain `communicate` step, so propably they wre closed becuse client did not replay or they were not valid

In [592]:
all_ir_with_time_communicate[all_ir_with_time_communicate["Resolved-communicate"].isnull()].head()

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,Severity,Resolution,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review,Resolved-communicate
0,CYTCI-10248111,7809422,Closed,yutaka.yasumi,2023-10-02 06:32:00,2023-10-12 00:00,2023-10-19 11:57:00,CMS Hasche Sigle,Incident Request (IR),False,3 - Medium,Rejected,17 days 05:25:00,7809422,8 days 19:47:00,8 days 09:38:00,8 days 09:38:00,NaT
1,CYTCI-9630,8042763,Closed,yutaka.yasumi,2023-11-27 09:08:00,2023-12-06 00:00,2023-11-29 10:14:00,Porsche,Incident Request (IR),False,3 - Medium,Rejected,2 days 01:06:00,8042763,0 days 00:00:00,2 days 01:06:00,2 days 01:06:00,NaT
10,CYTCI-9454,7904707,Closed,yutaka.yasumi,2023-10-26 11:10:00,2023-10-31 00:00,2023-11-01 07:23:00,Vaillant,Incident Request (IR),False,2 - Serious,Rejected,5 days 20:13:00,7904707,4 days 18:53:00,1 days 01:20:00,1 days 01:09:00,NaT
19,CYTCI-9323,7654712,Closed,msalajan,2023-09-27 10:18:00,2023-09-26 00:00,2023-09-27 21:42:00,Numiga,Incident Request (IR),False,2 - Serious,Rejected,0 days 11:24:00,7654712,0 days 07:28:00,0 days 03:56:00,0 days 00:00:00,NaT
20,CYTCI-9235,7654624,Closed,nsanchezruiz,2023-09-19 15:55:00,2023-09-24 00:00,2023-10-18 12:09:00,Deutschlandradio,Incident Request (IR),False,3 - Medium,Rejected,28 days 20:14:00,7654624,26 days 06:02:00,2 days 14:12:00,1 days 20:46:00,NaT


Check for null values

In [593]:
all_ir_with_time_communicate.isnull().sum()

Issue key                    0
Issue id                     0
Status                       0
Reporter                     0
Created                      0
Due Date                     0
Resolved                     0
Customer                     0
Type of Request              0
Is Escalated                 0
Severity                     0
Resolution                   0
ticket_age                   0
Parent id                    0
time_outside_codebusters     0
time_on_codebusters          0
time_in_review              10
Resolved-communicate        59
dtype: int64

### Calculate ticket age base on `communicate` step start date

In [594]:
all_ir_with_time_communicate["Ticket_age_communicate_date"] = all_ir_with_time_communicate["Resolved-communicate"] - all_ir_with_time_communicate["Created"]
all_ir_with_time_communicate

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,Severity,Resolution,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review,Resolved-communicate,Ticket_age_communicate_date
0,CYTCI-10248111,7809422,Closed,yutaka.yasumi,2023-10-02 06:32:00,2023-10-12 00:00,2023-10-19 11:57:00,CMS Hasche Sigle,Incident Request (IR),False,3 - Medium,Rejected,17 days 05:25:00,7809422,8 days 19:47:00,8 days 09:38:00,8 days 09:38:00,NaT,NaT
1,CYTCI-9630,8042763,Closed,yutaka.yasumi,2023-11-27 09:08:00,2023-12-06 00:00,2023-11-29 10:14:00,Porsche,Incident Request (IR),False,3 - Medium,Rejected,2 days 01:06:00,8042763,0 days 00:00:00,2 days 01:06:00,2 days 01:06:00,NaT,NaT
2,CYTCI-9627,8041529,Closed,yutaka.yasumi,2023-11-27 03:51:00,2023-12-07 00:00,2023-11-30 08:27:00,Rheinmetall,Incident Request (IR),False,3 - Medium,Done,3 days 04:36:00,8041529,0 days 17:09:00,2 days 11:27:00,2 days 06:31:00,2023-11-29 15:18:00,2 days 11:27:00
3,CYTCI-9625,8035785,Closed,nsanchezruiz,2023-11-24 12:33:00,2023-11-30 00:00,2023-11-28 15:35:00,Egis,Incident Request (IR),False,3 - Medium Urgent,Done,4 days 03:02:00,8035785,0 days 05:24:00,3 days 21:37:00,3 days 04:11:00,2023-11-28 10:11:00,3 days 21:38:00
4,CYTCI-9552,7981834,Closed,nsanchezruiz,2023-11-14 14:58:00,2023-11-27 00:00,2023-11-29 10:03:00,MFTBC,Incident Request (IR),False,3 - Medium,Done,14 days 19:05:00,7981834,1 days 22:00:00,12 days 21:05:00,1 days 01:23:00,2023-11-28 09:03:00,13 days 18:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,CYTCI-6089,7651525,Closed,yutaka.yasumi,2021-12-06 05:16:00,2021-12-12 00:00,2022-01-17 01:05:00,Apleona,Incident Request (IR),False,3 - Medium,Done,41 days 19:49:00,7651525,28 days 05:20:00,13 days 14:29:00,0 days 09:51:00,2021-12-21 10:48:00,15 days 05:32:00
279,CYTCI-6059,7651495,Closed,yutaka.yasumi,2021-11-25 05:05:00,2021-11-29 00:00,2022-01-17 02:41:00,Hensoldt,Incident Request (IR),False,3 - Medium,Rejected,52 days 21:36:00,7651495,14 days 18:27:00,38 days 03:09:00,6 days 18:15:00,NaT,NaT
280,CYTCI-6058,7651494,Closed,ssanz,2021-11-24 17:21:00,2021-11-29 00:00,2022-02-22 12:55:00,Allianz,Incident Request (IR),False,4 - Minor,Rejected,89 days 19:34:00,7651494,0 days 00:00:00,89 days 19:35:00,5 days 21:19:00,NaT,NaT
281,CYTCI-5913,7651349,Closed,yutaka.yasumi,2021-10-28 13:45:00,2022-01-20 00:00,2022-01-17 11:26:00,Rheinmetall,Incident Request (IR),False,4 - Minor,Rejected,80 days 21:41:00,7651349,39 days 14:22:00,41 days 07:19:00,7 days 20:58:00,2021-11-26 09:05:00,28 days 19:20:00


### Drop rows which contain null value except column `Resolved-communicate` and `ticket_age_communicate_date`

In [595]:
all_ir_with_time_without_null = all_ir_with_time_communicate.dropna(subset=all_ir_with_time_communicate.columns.difference(['Resolved-communicate', 'Ticket_age_communicate_date']))
all_ir_with_time_without_null.isnull().sum()

Issue key                       0
Issue id                        0
Status                          0
Reporter                        0
Created                         0
Due Date                        0
Resolved                        0
Customer                        0
Type of Request                 0
Is Escalated                    0
Severity                        0
Resolution                      0
ticket_age                      0
Parent id                       0
time_outside_codebusters        0
time_on_codebusters             0
time_in_review                  0
Resolved-communicate           57
Ticket_age_communicate_date    57
dtype: int64

In [596]:
all_ir_with_time_without_null.shape

(273, 19)

In [597]:
all_ir_with_time_without_null.info()

<class 'pandas.core.frame.DataFrame'>
Index: 273 entries, 0 to 282
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype          
---  ------                       --------------  -----          
 0   Issue key                    273 non-null    object         
 1   Issue id                     273 non-null    int64          
 2   Status                       273 non-null    object         
 3   Reporter                     273 non-null    object         
 4   Created                      273 non-null    datetime64[ns] 
 5   Due Date                     273 non-null    object         
 6   Resolved                     273 non-null    datetime64[ns] 
 7   Customer                     273 non-null    object         
 8   Type of Request              273 non-null    object         
 9   Is Escalated                 273 non-null    bool           
 10  Severity                     273 non-null    object         
 11  Resolution                   273 non-

## Calculate Compliance Ratio

In [598]:

# get to limit for each severity
# check the days diff between creation and resolved
# compare limit with diff
# if diff > limit then set overlimit to true
all_ir_with_time_sla = all_ir_with_time_without_null
all_ir_with_time_sla['overlimit'] = all_ir_with_time_sla.apply(lambda row: verify_sla_limit(row, "ticket_age"), axis=1)
all_ir_with_time_sla_communicate = all_ir_with_time_sla['overlimit_communicate'] = all_ir_with_time_sla.apply(lambda row: verify_sla_limit(row, "Ticket_age_communicate_date"), axis=1)        
all_ir_with_time_sla 
# all_ir_escalated_match.apply(lambda row: find_escalatetions(row, all_ir_escalated_match), axis=1)
# ticket_age_comminicate_date

    

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\2679089034.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir_with_time_sla['overlimit'] = all_ir_with_time_sla.apply(lambda row: verify_sla_limit(row, "ticket_age"), axis=1)
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\2679089034.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir_with_time_sla_communicate = all_ir_with_time_sla['overlimit_communicate'] = all_ir_with_time_sla.apply(lambda row: verify_sla_limit(row, "Ticket_age_co

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,...,Resolution,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review,Resolved-communicate,Ticket_age_communicate_date,overlimit,overlimit_communicate
0,CYTCI-10248111,7809422,Closed,yutaka.yasumi,2023-10-02 06:32:00,2023-10-12 00:00,2023-10-19 11:57:00,CMS Hasche Sigle,Incident Request (IR),False,...,Rejected,17 days 05:25:00,7809422,8 days 19:47:00,8 days 09:38:00,8 days 09:38:00,NaT,NaT,True,False
1,CYTCI-9630,8042763,Closed,yutaka.yasumi,2023-11-27 09:08:00,2023-12-06 00:00,2023-11-29 10:14:00,Porsche,Incident Request (IR),False,...,Rejected,2 days 01:06:00,8042763,0 days 00:00:00,2 days 01:06:00,2 days 01:06:00,NaT,NaT,False,False
2,CYTCI-9627,8041529,Closed,yutaka.yasumi,2023-11-27 03:51:00,2023-12-07 00:00,2023-11-30 08:27:00,Rheinmetall,Incident Request (IR),False,...,Done,3 days 04:36:00,8041529,0 days 17:09:00,2 days 11:27:00,2 days 06:31:00,2023-11-29 15:18:00,2 days 11:27:00,False,False
3,CYTCI-9625,8035785,Closed,nsanchezruiz,2023-11-24 12:33:00,2023-11-30 00:00,2023-11-28 15:35:00,Egis,Incident Request (IR),False,...,Done,4 days 03:02:00,8035785,0 days 05:24:00,3 days 21:37:00,3 days 04:11:00,2023-11-28 10:11:00,3 days 21:38:00,False,False
4,CYTCI-9552,7981834,Closed,nsanchezruiz,2023-11-14 14:58:00,2023-11-27 00:00,2023-11-29 10:03:00,MFTBC,Incident Request (IR),False,...,Done,14 days 19:05:00,7981834,1 days 22:00:00,12 days 21:05:00,1 days 01:23:00,2023-11-28 09:03:00,13 days 18:05:00,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,CYTCI-6089,7651525,Closed,yutaka.yasumi,2021-12-06 05:16:00,2021-12-12 00:00,2022-01-17 01:05:00,Apleona,Incident Request (IR),False,...,Done,41 days 19:49:00,7651525,28 days 05:20:00,13 days 14:29:00,0 days 09:51:00,2021-12-21 10:48:00,15 days 05:32:00,True,True
279,CYTCI-6059,7651495,Closed,yutaka.yasumi,2021-11-25 05:05:00,2021-11-29 00:00,2022-01-17 02:41:00,Hensoldt,Incident Request (IR),False,...,Rejected,52 days 21:36:00,7651495,14 days 18:27:00,38 days 03:09:00,6 days 18:15:00,NaT,NaT,True,False
280,CYTCI-6058,7651494,Closed,ssanz,2021-11-24 17:21:00,2021-11-29 00:00,2022-02-22 12:55:00,Allianz,Incident Request (IR),False,...,Rejected,89 days 19:34:00,7651494,0 days 00:00:00,89 days 19:35:00,5 days 21:19:00,NaT,NaT,True,False
281,CYTCI-5913,7651349,Closed,yutaka.yasumi,2021-10-28 13:45:00,2022-01-20 00:00,2022-01-17 11:26:00,Rheinmetall,Incident Request (IR),False,...,Rejected,80 days 21:41:00,7651349,39 days 14:22:00,41 days 07:19:00,7 days 20:58:00,2021-11-26 09:05:00,28 days 19:20:00,True,False


## Convert `time in..` columns to minutes

Convert `timedelta` columns to float

In [599]:
all_ir_with_time_converted = all_ir_with_time_sla
all_ir_with_time_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 273 entries, 0 to 282
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype          
---  ------                       --------------  -----          
 0   Issue key                    273 non-null    object         
 1   Issue id                     273 non-null    int64          
 2   Status                       273 non-null    object         
 3   Reporter                     273 non-null    object         
 4   Created                      273 non-null    datetime64[ns] 
 5   Due Date                     273 non-null    object         
 6   Resolved                     273 non-null    datetime64[ns] 
 7   Customer                     273 non-null    object         
 8   Type of Request              273 non-null    object         
 9   Is Escalated                 273 non-null    bool           
 10  Severity                     273 non-null    object         
 11  Resolution                   273 non-

Convert `time in..` columns to minutes

In [600]:
all_ir_with_time_converted['time_in_review'] = all_ir_with_time_converted['time_in_review'].dt.total_seconds().div(60)
all_ir_with_time_converted['time_on_codebusters'] = all_ir_with_time_converted['time_on_codebusters'].dt.total_seconds().div(60)
all_ir_with_time_converted['time_outside_codebusters'] = all_ir_with_time_converted['time_outside_codebusters'].dt.total_seconds().div(60)
all_ir_with_time_converted['ticket_age'] = all_ir_with_time_converted['ticket_age'].dt.total_seconds().div(60)

all_ir_with_time_converted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 273 entries, 0 to 282
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype          
---  ------                       --------------  -----          
 0   Issue key                    273 non-null    object         
 1   Issue id                     273 non-null    int64          
 2   Status                       273 non-null    object         
 3   Reporter                     273 non-null    object         
 4   Created                      273 non-null    datetime64[ns] 
 5   Due Date                     273 non-null    object         
 6   Resolved                     273 non-null    datetime64[ns] 
 7   Customer                     273 non-null    object         
 8   Type of Request              273 non-null    object         
 9   Is Escalated                 273 non-null    bool           
 10  Severity                     273 non-null    object         
 11  Resolution                   273 non-

C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\3678482499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir_with_time_converted['time_in_review'] = all_ir_with_time_converted['time_in_review'].dt.total_seconds().div(60)
C:\Users\gmalarski\AppData\Local\Temp\ipykernel_21008\3678482499.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_ir_with_time_converted['time_on_codebusters'] = all_ir_with_time_converted['time_on_codebusters'].dt.total_seconds().div(60)
C:\Users\gmalarski\AppData\Loc

In [601]:
all_ir_with_time_converted.reset_index(inplace=True)
all_ir_with_time_converted

,index,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,...,Resolution,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review,Resolved-communicate,Ticket_age_communicate_date,overlimit,overlimit_communicate
0,0,CYTCI-10248111,7809422,Closed,yutaka.yasumi,2023-10-02 06:32:00,2023-10-12 00:00,2023-10-19 11:57:00,CMS Hasche Sigle,Incident Request (IR),...,Rejected,24805.0,7809422,12707.0,12098.0,12098.0,NaT,NaT,True,False
1,1,CYTCI-9630,8042763,Closed,yutaka.yasumi,2023-11-27 09:08:00,2023-12-06 00:00,2023-11-29 10:14:00,Porsche,Incident Request (IR),...,Rejected,2946.0,8042763,0.0,2946.0,2946.0,NaT,NaT,False,False
2,2,CYTCI-9627,8041529,Closed,yutaka.yasumi,2023-11-27 03:51:00,2023-12-07 00:00,2023-11-30 08:27:00,Rheinmetall,Incident Request (IR),...,Done,4596.0,8041529,1029.0,3567.0,3271.0,2023-11-29 15:18:00,2 days 11:27:00,False,False
3,3,CYTCI-9625,8035785,Closed,nsanchezruiz,2023-11-24 12:33:00,2023-11-30 00:00,2023-11-28 15:35:00,Egis,Incident Request (IR),...,Done,5942.0,8035785,324.0,5617.0,4571.0,2023-11-28 10:11:00,3 days 21:38:00,False,False
4,4,CYTCI-9552,7981834,Closed,nsanchezruiz,2023-11-14 14:58:00,2023-11-27 00:00,2023-11-29 10:03:00,MFTBC,Incident Request (IR),...,Done,21305.0,7981834,2760.0,18545.0,1523.0,2023-11-28 09:03:00,13 days 18:05:00,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,278,CYTCI-6089,7651525,Closed,yutaka.yasumi,2021-12-06 05:16:00,2021-12-12 00:00,2022-01-17 01:05:00,Apleona,Incident Request (IR),...,Done,60229.0,7651525,40640.0,19589.0,591.0,2021-12-21 10:48:00,15 days 05:32:00,True,True
269,279,CYTCI-6059,7651495,Closed,yutaka.yasumi,2021-11-25 05:05:00,2021-11-29 00:00,2022-01-17 02:41:00,Hensoldt,Incident Request (IR),...,Rejected,76176.0,7651495,21267.0,54909.0,9735.0,NaT,NaT,True,False
270,280,CYTCI-6058,7651494,Closed,ssanz,2021-11-24 17:21:00,2021-11-29 00:00,2022-02-22 12:55:00,Allianz,Incident Request (IR),...,Rejected,129334.0,7651494,0.0,129335.0,8479.0,NaT,NaT,True,False
271,281,CYTCI-5913,7651349,Closed,yutaka.yasumi,2021-10-28 13:45:00,2022-01-20 00:00,2022-01-17 11:26:00,Rheinmetall,Incident Request (IR),...,Rejected,116501.0,7651349,57022.0,59479.0,11338.0,2021-11-26 09:05:00,28 days 19:20:00,True,False


In [607]:
all_ir_with_time_converted.isnull().sum()

index                           0
Issue key                       0
Issue id                        0
Status                          0
Reporter                        0
Created                         0
Due Date                        0
Resolved                        0
Customer                        0
Type of Request                 0
Is Escalated                    0
Severity                        0
Resolution                      0
ticket_age                      0
Parent id                       0
time_outside_codebusters        0
time_on_codebusters             0
time_in_review                  0
Resolved-communicate           57
Ticket_age_communicate_date    57
overlimit                       0
overlimit_communicate           0
dtype: int64

In [608]:
all_ir_with_time_converted_communicate = all_ir_with_time_converted.dropna()
all_ir_with_time_converted_communicate

,index,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,...,Resolution,ticket_age,Parent id,time_outside_codebusters,time_on_codebusters,time_in_review,Resolved-communicate,Ticket_age_communicate_date,overlimit,overlimit_communicate
2,2,CYTCI-9627,8041529,Closed,yutaka.yasumi,2023-11-27 03:51:00,2023-12-07 00:00,2023-11-30 08:27:00,Rheinmetall,Incident Request (IR),...,Done,4596.0,8041529,1029.0,3567.0,3271.0,2023-11-29 15:18:00,2 days 11:27:00,False,False
3,3,CYTCI-9625,8035785,Closed,nsanchezruiz,2023-11-24 12:33:00,2023-11-30 00:00,2023-11-28 15:35:00,Egis,Incident Request (IR),...,Done,5942.0,8035785,324.0,5617.0,4571.0,2023-11-28 10:11:00,3 days 21:38:00,False,False
4,4,CYTCI-9552,7981834,Closed,nsanchezruiz,2023-11-14 14:58:00,2023-11-27 00:00,2023-11-29 10:03:00,MFTBC,Incident Request (IR),...,Done,21305.0,7981834,2760.0,18545.0,1523.0,2023-11-28 09:03:00,13 days 18:05:00,True,True
5,5,CYTCI-9522,7957419,Closed,ssanz,2023-11-08 17:16:00,2023-11-17 00:00,2023-11-29 16:26:00,Rheinmetall,Incident Request (IR),...,Done,30190.0,7957419,14711.0,15479.0,11037.0,2023-11-24 09:51:00,15 days 16:35:00,True,True
6,6,CYTCI-9521,7957088,Closed,ssanz,2023-11-08 16:10:00,2023-11-10 00:00,2023-11-13 09:32:00,Numiga,Incident Request (IR),...,Done,6802.0,7957088,6586.0,216.0,8.0,2023-11-09 10:44:00,0 days 18:34:00,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,274,CYTCI-6184,7651619,Closed,ssanz,2021-12-27 14:21:00,2022-01-03 00:00,2022-03-09 14:02:00,Allianz,Incident Request (IR),...,Done,103661.0,7651619,80893.0,22768.0,11605.0,2022-03-08 09:09:00,70 days 18:48:00,True,True
265,275,CYTCI-6157,7651592,Closed,nsanchezruiz,2021-12-21 15:11:00,2021-12-29 00:00,2022-01-05 15:38:00,CEPSA,Incident Request (IR),...,Done,21627.0,7651592,88.0,21539.0,21461.0,2022-01-05 15:27:00,15 days 00:16:00,True,True
267,277,CYTCI-6090,7651526,Closed,debert,2021-12-06 12:25:00,2021-12-30 00:00,2022-02-04 14:38:00,OBB,Incident Request (IR),...,Done,86533.0,7651526,2881.0,84765.0,4486.0,2022-02-02 14:37:00,58 days 02:12:00,True,True
268,278,CYTCI-6089,7651525,Closed,yutaka.yasumi,2021-12-06 05:16:00,2021-12-12 00:00,2022-01-17 01:05:00,Apleona,Incident Request (IR),...,Done,60229.0,7651525,40640.0,19589.0,591.0,2021-12-21 10:48:00,15 days 05:32:00,True,True


In [602]:
all_ir_with_time_converted['Severity'].unique()

array(['3 - Medium', '3 - Medium Urgent', '2 - Serious',
       '2 - Serious Urgent', '1 - Critical', '4 - Minor'], dtype=object)

In [603]:
file_path = Path('../../DataSets/KPIs-final/all_ir_kpi.csv')
all_ir_with_time_converted.to_csv(path_or_buf=file_path)

In [610]:
file_path = Path('../../DataSets/KPIs-final/all_ir_kpi_communicate.csv')
all_ir_with_time_converted_communicate.to_csv(path_or_buf=file_path)

# Investigation starts

In [604]:
all_individual_tasks_reivew[all_individual_tasks_reivew["Parent id"] == 438360]

,index,Summary,Issue id,Parent id,Created,Resolved,time_in_review
3401,18969,Review,438363,438360,2022-12-29 11:45:00,2022-12-29 12:21:00,0 days 00:36:00
3402,18971,Review,438361,438360,2022-12-29 11:30:00,2022-12-29 11:42:00,0 days 00:12:00


In [605]:
all_ir[all_ir["Issue id"] == 438360]

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,Severity,Resolution,ticket_age


In [606]:
null_rows

,Issue key,Issue id,Status,Reporter,Created,Due Date,Resolved,Customer,Type of Request,Is Escalated,Severity,Resolution,ticket_age,Parent id,time_in_review
136,CYTCI-7932,7653337,Closed,nsanchezruiz,2022-12-27 14:47:00,2023-01-02 00:00,2022-12-27 14:52:00,ProcurX,Incident Request (IR),False,3 - Medium,Rejected,0 days 00:05:00,NaN,NaT
138,CYTCI-7929,7653334,Closed,nsanchezruiz,2022-12-23 10:06:00,2023-01-02 00:00,2022-12-27 10:49:00,Endress Hauser,Incident Request (IR),False,3 - Medium,Rejected,4 days 00:43:00,NaN,NaT
140,CYTCI-7921,7653326,Closed,yutaka.yasumi,2022-12-22 10:09:00,2022-12-25 00:00,2022-12-30 10:21:00,Lufthansa City Center,Incident Request (IR),False,3 - Medium,Done,8 days 00:12:00,NaN,NaT
142,CYTCI-7887,7653292,Closed,nsanchezruiz,2022-12-16 14:40:00,2022-12-20 00:00,2023-01-02 08:59:00,Osram,Incident Request (IR),False,3 - Medium,Done,16 days 18:19:00,NaN,NaT
143,CYTCI-7885,7653290,Closed,nsanchezruiz,2022-12-16 13:35:00,2022-12-21 00:00,2022-12-21 13:51:00,OBB,Incident Request (IR),False,3 - Medium,Done,5 days 00:16:00,NaN,NaT
144,CYTCI-7865,7653270,Closed,nsanchezruiz,2022-12-14 11:30:00,2022-12-19 00:00,2022-12-21 09:17:00,Bentley,Incident Request (IR),False,3 - Medium,Done,6 days 21:47:00,NaN,NaT
145,CYTCI-7861,7653266,Closed,ssanz,2022-12-13 15:15:00,2022-12-19 00:00,2023-03-01 15:56:00,Audi,Incident Request (IR),False,3 - Medium,Done,78 days 00:41:00,NaN,NaT
146,CYTCI-7836,7653242,Closed,yutaka.yasumi,2022-12-07 08:47:00,2022-12-07 00:00,2022-12-16 10:51:00,Endress Hauser,Incident Request (IR),False,3 - Medium,Done,9 days 02:04:00,NaN,NaT
147,CYTCI-7816,7653224,Closed,yutaka.yasumi,2022-12-05 10:09:00,2022-12-08 00:00,2023-03-03 14:03:00,CEPSA,Incident Request (IR),False,3 - Medium,Done,88 days 03:54:00,NaN,NaT
148,CYTCI-7808,7653221,Closed,yutaka.yasumi,2022-12-05 08:05:00,2022-12-08 00:00,2023-01-04 02:16:00,Amadeus,Incident Request (IR),False,3 - Medium,Done,29 days 18:11:00,NaN,NaT
